<a href="https://colab.research.google.com/github/PythonJulianPrieto/CienciaDeDatos/blob/main/UsoDeSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar -xvf /content/spark-3.2.0-bin-hadoop3.2.tgz
import os

os.environ['SPARK_HOME'] = '/content/spark-3.2.0-bin-hadoop3.2'
!pip install -q findspark

import findspark

findspark.init()



#***Acoplar Spark y configurar la sesion de SPARKSQL***

In [ ]:
#Configurar la sesión para Spark-SQ
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master('local[*]').appName('Ejercicio').getOrCreate()

#***USAR DATAFRAMES EN SPARK***#

***Pequeño ejemplo de dataframes y pandas.***

este ejercicio solo aplica para el dataframe de pandas

In [ ]:
#Importar las librerias de pandas
import pandas as pd

In [ ]:
#serie = pd.Series(data,index = index)

curso = pd.Series(['Julian', 'Alfredo', 'Maria', 'Danna', 'Carlos'],index = [1,2,3,4,5])


In [ ]:
print(curso)
type(curso)

1     Julian
2    Alfredo
3      Maria
4      Danna
5     Carlos
dtype: object


pandas.core.series.Series

#***DATAFRAMES DE SPARK***

Mantiene el concepto de DataFrames de pandas, pero la esctructura es un ***RDD*** de spark

podemos crear un dataframe de un archivo, una consulta, un dataworkhouse, Json, Generar almacenamiento.



##Creacicon del DataFrame desde listas o arreglos en el framework de trabajo

In [ ]:
calificaciones = [('Julian',4.6),('pepe', 5.0),('Alfredo',3.4),('Miguel', 4.7,),('martha',2.5)]
print(calificaciones)
type(calificaciones)

[('Julian', 4.6), ('pepe', 5.0), ('Alfredo', 3.4), ('Miguel', 4.7), ('martha', 2.5)]


list

In [ ]:
#Convertimos la lista de pandas (Python) a un DataFrame
df = spark.createDataFrame(calificaciones,["Nombre", "Notas"])


In [ ]:
df.show()

+-------+-----+
| Nombre|Notas|
+-------+-----+
| Julian|  4.6|
|   pepe|  5.0|
|Alfredo|  3.4|
| Miguel|  4.7|
| martha|  2.5|
+-------+-----+



In [ ]:
df.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Notas: double (nullable = true)



In [ ]:
calificaciones2 = [('Julian',4.6),('pepe', 5.0),('Alfredo',0.0),('Miguel', 4.7,),('martha',2.5)]

In [ ]:
df2 = spark.createDataFrame(calificaciones2,["Nombre", "Notas"])

In [ ]:
df2.show()

+-------+-----+
| Nombre|Notas|
+-------+-----+
| Julian|  4.6|
|   pepe|  5.0|
|Alfredo|  0.0|
| Miguel|  4.7|
| martha|  2.5|
+-------+-----+



In [ ]:
type(df.schema)

pyspark.sql.types.StructType

In [ ]:
from pyspark.sql.types import *
ejemploSchema = StructType([StructField('Estudiante',StringType(),True),StructField('Notas',FloatType(),True)])

In [ ]:
calificaciones3 = [('Julian',4.6),('pepe', 5.0),('Alfredo',0.0),('Miguel', 4.7,),('martha',2.5)]
df3 = spark.createDataFrame(calificaciones3,ejemploSchema)

In [ ]:
df3.show()
df3.printSchema()

+----------+-----+
|Estudiante|Notas|
+----------+-----+
|    Julian|  4.6|
|      pepe|  5.0|
|   Alfredo|  0.0|
|    Miguel|  4.7|
|    martha|  2.5|
+----------+-----+

root
 |-- Estudiante: string (nullable = true)
 |-- Notas: float (nullable = true)



#***CARGAR DATAFRAME SPARK DESDE ARCHIVOS***

In [ ]:
!wget https://raw.githubusercontent.com/adiacla/bigdata/master/Churn_Modelling.csv

--2021-12-29 20:00:53--  https://raw.githubusercontent.com/adiacla/bigdata/master/Churn_Modelling.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘Churn_Modelling.csv’

Churn_Modelling.csv 100%[===================>] 668.81K  --.-KB/s    in 0.05s   

2021-12-29 20:00:54 (11.9 MB/s) - ‘Churn_Modelling.csv’ saved [684858/684858]



In [ ]:
churnDF = spark.read.csv('Churn_Modelling.csv',header=True,inferSchema=True)

In [ ]:
churnDF.take(2)

[Row(RowNumber=1, CustomerId=15634602, Surname='Hargrave', CreditScore=619, Geography='France', Gender='Female', Age=42, Tenure=2, Balance=0.0, NumOfProducts=1, HasCrCard=1, IsActiveMember=1, EstimatedSalary=101348.88, Exited=1),
 Row(RowNumber=2, CustomerId=15647311, Surname='Hill', CreditScore=608, Geography='Spain', Gender='Female', Age=41, Tenure=1, Balance=83807.86, NumOfProducts=1, HasCrCard=0, IsActiveMember=1, EstimatedSalary=112542.58, Exited=0)]

In [ ]:
churnDF.show(n=3)

+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|     0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1|83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8|159660.8|            3|        1|             0|      113931.57|     1|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+

In [ ]:
churnDF.show(2)

+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|     0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1|83807.86|            1|        0|             1|      112542.58|     0|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
only showing top 2 rows



In [ ]:
churnDF.count()

10000

In [ ]:
churnDF.show(n=9995)

In [ ]:
churnDF.show(vertical=True)

In [ ]:
churnDF.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [ ]:
churnDF.filter(churnDF['CreditScore']>600).show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|     0|
|        5|  15737888| Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|        1|             1|        7

#***CREAR UN DATAFRAME USANDO ARCHIVOS TIPO JSON***

In [ ]:
!wget https://raw.githubusercontent.com/adiacla/bigdata/master/CovidList.json


--2021-12-29 20:28:43--  https://raw.githubusercontent.com/adiacla/bigdata/master/CovidList.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1845911 (1.8M) [text/plain]
Saving to: ‘CovidList.json’

CovidList.json      100%[===================>]   1.76M  --.-KB/s    in 0.07s   

2021-12-29 20:28:43 (25.1 MB/s) - ‘CovidList.json’ saved [1845911/1845911]



In [ ]:
!cat CovidList.json


In [ ]:
covid = spark.read.json('CovidList.json')

In [ ]:
covid.printSchema()

root
 |-- cases: long (nullable = true)
 |-- continentExp: string (nullable = true)
 |-- countriesAndTerritories: string (nullable = true)
 |-- countryterritoryCode: string (nullable = true)
 |-- dateRep: string (nullable = true)
 |-- day: string (nullable = true)
 |-- deaths: long (nullable = true)
 |-- geoId: string (nullable = true)
 |-- month: string (nullable = true)
 |-- popData2020: string (nullable = true)
 |-- year: long (nullable = true)



In [ ]:
covid.show(10)

+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----+-----------+----+
|cases|continentExp|countriesAndTerritories|countryterritoryCode|   dateRep|day|deaths|geoId|month|popData2020|year|
+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----+-----------+----+
| 3314|        NULL|                Austria|                 AUT|15/12/2021| 15|    59|     |   12|    8901064|2021|
| 2829|        NULL|                Austria|                 AUT|14/12/2021| 14|    28|     |   12|    8901064|2021|
| 3591|      Europe|                Austria|                 AUT|13/12/2021| 13|    19|   AT|   12|    8901064|2021|
| 4909|      Europe|                Austria|                 AUT|12/12/2021| 12|    53|   AT|   12|    8901064|2021|
| 2512|      Europe|                Austria|                 AUT|11/12/2021| 11|   110|   AT|   12|    8901064|2021|
| 4223|      Europe|                Austria|                 AUT

In [ ]:
covid.count()

8700

In [ ]:
type(covid)

pyspark.sql.dataframe.DataFrame

In [ ]:
#Contar cuantas particiones tiene
covid.rdd.getNumPartitions()

1

In [ ]:
#crear filtros por columna
muertes = covid.select('countriesAndTerritories','deaths')
muertes.show()

+-----------------------+------+
|countriesAndTerritories|deaths|
+-----------------------+------+
|                Austria|    59|
|                Austria|    28|
|                Austria|    19|
|                Austria|    53|
|                Austria|   110|
|                Austria|    17|
|                Austria|    47|
|                Austria|    67|
|                Austria|    69|
|                Austria|    59|
|                Austria|    85|
|                Austria|    89|
|                Austria|    76|
|                Austria|    56|
|                Austria|    77|
|                Austria|    36|
|                Austria|    43|
|                Austria|    39|
|                Austria|    44|
|                Austria|    66|
+-----------------------+------+
only showing top 20 rows



In [ ]:
covidSinColumna = covid.drop('geoId')

In [ ]:
covidSinColumna.printSchema()

root
 |-- cases: long (nullable = true)
 |-- continentExp: string (nullable = true)
 |-- countriesAndTerritories: string (nullable = true)
 |-- countryterritoryCode: string (nullable = true)
 |-- dateRep: string (nullable = true)
 |-- day: string (nullable = true)
 |-- deaths: long (nullable = true)
 |-- month: string (nullable = true)
 |-- popData2020: string (nullable = true)
 |-- year: long (nullable = true)



In [ ]:
covid.printSchema()

root
 |-- cases: long (nullable = true)
 |-- continentExp: string (nullable = true)
 |-- countriesAndTerritories: string (nullable = true)
 |-- countryterritoryCode: string (nullable = true)
 |-- dateRep: string (nullable = true)
 |-- day: string (nullable = true)
 |-- deaths: long (nullable = true)
 |-- geoId: string (nullable = true)
 |-- month: string (nullable = true)
 |-- popData2020: string (nullable = true)
 |-- year: long (nullable = true)



In [ ]:
covidSinColumna.show(10)

+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----------+----+
|cases|continentExp|countriesAndTerritories|countryterritoryCode|   dateRep|day|deaths|month|popData2020|year|
+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----------+----+
| 3314|        NULL|                Austria|                 AUT|15/12/2021| 15|    59|   12|    8901064|2021|
| 2829|        NULL|                Austria|                 AUT|14/12/2021| 14|    28|   12|    8901064|2021|
| 3591|      Europe|                Austria|                 AUT|13/12/2021| 13|    19|   12|    8901064|2021|
| 4909|      Europe|                Austria|                 AUT|12/12/2021| 12|    53|   12|    8901064|2021|
| 2512|      Europe|                Austria|                 AUT|11/12/2021| 11|   110|   12|    8901064|2021|
| 4223|      Europe|                Austria|                 AUT|10/12/2021| 10|    17|   12|    8901064|2021|
|

#***BORRADO O FILTRADO DE FILAS***


In [ ]:
filtradoFilasNull = covidSinColumna.filter(covidSinColumna['continentExp'] =='NULL')

In [ ]:
filtradoFilasNull.show()

+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----------+----+
|cases|continentExp|countriesAndTerritories|countryterritoryCode|   dateRep|day|deaths|month|popData2020|year|
+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----------+----+
| 3314|        NULL|                Austria|                 AUT|15/12/2021| 15|    59|   12|    8901064|2021|
| 2829|        NULL|                Austria|                 AUT|14/12/2021| 14|    28|   12|    8901064|2021|
+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----------+----+



In [ ]:
filtradoFilasNull.count()

2

In [ ]:
#Guardadndo en otro RDD SIN  las dos filas con NULL
covidSinNull = covidSinColumna.filter(covidSinColumna['continentExp'] !='NULL')
covidSinNull.count()

8698

In [ ]:
covid.show()

+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----+-----------+----+
|cases|continentExp|countriesAndTerritories|countryterritoryCode|   dateRep|day|deaths|geoId|month|popData2020|year|
+-----+------------+-----------------------+--------------------+----------+---+------+-----+-----+-----------+----+
| 3314|        NULL|                Austria|                 AUT|15/12/2021| 15|    59|     |   12|    8901064|2021|
| 2829|        NULL|                Austria|                 AUT|14/12/2021| 14|    28|     |   12|    8901064|2021|
| 3591|      Europe|                Austria|                 AUT|13/12/2021| 13|    19|   AT|   12|    8901064|2021|
| 4909|      Europe|                Austria|                 AUT|12/12/2021| 12|    53|   AT|   12|    8901064|2021|
| 2512|      Europe|                Austria|                 AUT|11/12/2021| 11|   110|   AT|   12|    8901064|2021|
| 4223|      Europe|                Austria|                 AUT

In [ ]:
##Muestre los registros diarios de los casos de mas de 10000 pacientes mostrando solo el pais, el caso y la fecha
registroDiario = covid.select('day','countriesAndTerritories','cases','dateRep').filter(covid['cases']>10000)

In [ ]:
registroDiario.show(n=771)

In [ ]:
import numpy as np
#Calcular 
total = np.sum(covidSinNull.select('cases').collect())
print(total)

50490812


In [ ]:
total2 = np.mean(covidSinNull.select('cases').collect())
print(total2)

5804.876063462865


In [ ]:
covidSinNull.sort(covidSinNull.cases.desc()).show()

+-------+------------+-----------------------+--------------------+----------+---+------+-----+-----------+----+
|  cases|continentExp|countriesAndTerritories|countryterritoryCode|   dateRep|day|deaths|month|popData2020|year|
+-------+------------+-----------------------+--------------------+----------+---+------+-----+-----------+----+
|3645305|      Europe|                 France|                 FRA|01/03/2021| 01| 86454|   03|   67320216|2021|
|3109689|      Europe|                  Spain|                 ESP|28/02/2021| 28| 69142|   02|   47332614|2021|
|2925265|      Europe|                  Italy|                 ITA|01/03/2021| 01| 97699|   03|   59641488|2021|
|2447068|      Europe|                Germany|                 DEU|01/03/2021| 01| 70105|   03|   83166711|2021|
|1706986|      Europe|                 Poland|                 POL|01/03/2021| 01| 43769|   03|   37958138|2021|
|1240721|      Europe|                Czechia|                 CZE|01/03/2021| 01| 21020|   03| 

Cambiar el tipo de dato en un dataframe

In [ ]:
from typing import cast
covidSinNull=covidSinNull.withColumn("day",covidSinNull["day"].cast("Integer"))

In [ ]:
#Calcular cual es el promedio de los casos presentados cada dia del mes
covidSinNull.groupBy('countriesAndTerritories').mean('cases').show()

+-----------------------+------------------+
|countriesAndTerritories|        avg(cases)|
+-----------------------+------------------+
|                 Sweden| 4271.341379310345|
|                Germany| 22805.96551724138|
|                 France|28738.389655172414|
|                 Greece|3491.9275862068966|
|               Slovakia| 4462.134482758621|
|                Belgium| 6786.806896551724|
|                Finland| 713.9068965517241|
|                  Malta|140.41724137931035|
|                Croatia| 2268.020689655172|
|                  Italy|18134.089655172414|
|              Lithuania|1709.2620689655173|
|                 Norway|1112.8172413793104|
|                  Spain|18503.889655172414|
|                Czechia| 8162.331034482759|
|                Denmark| 1967.248275862069|
|                Ireland|2179.9206896551723|
|          Liechtenstein|18.868965517241378|
|                Iceland| 67.95862068965518|
|                 Cyprus| 490.4413793103448|
|         

In [ ]:
covidSinNull.printSchema()

root
 |-- cases: long (nullable = true)
 |-- continentExp: string (nullable = true)
 |-- countriesAndTerritories: string (nullable = true)
 |-- countryterritoryCode: string (nullable = true)
 |-- dateRep: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- deaths: long (nullable = true)
 |-- month: string (nullable = true)
 |-- popData2020: string (nullable = true)
 |-- year: long (nullable = true)



In [ ]:
#Total de los casos 2021 por pais
covidSinNull.filter(covidSinNull['year']==2021).groupBy('countriesAndTerritories').sum('cases').sort('sum(cases)').show()

+-----------------------+----------+
|countriesAndTerritories|sum(cases)|
+-----------------------+----------+
|          Liechtenstein|      5472|
|                Iceland|     19708|
|                  Malta|     40721|
|             Luxembourg|     94511|
|                 Cyprus|    142228|
|                Finland|    207033|
|                Estonia|    229869|
|                 Latvia|    263145|
|                 Norway|    322717|
|               Slovenia|    443873|
|              Lithuania|    495686|
|                Denmark|    570502|
|                Ireland|    632177|
|                Croatia|    657726|
|               Bulgaria|    718651|
|                 Greece|   1012659|
|               Portugal|   1200193|
|                Hungary|   1208020|
|                Austria|   1226443|
|                 Sweden|   1238689|
+-----------------------+----------+
only showing top 20 rows



In [ ]:
covidSinNull.filter(covidSinNull['year']==2021).filter(covidSinNull['continentExp']=='Europe').groupBy('countriesAndTerritories').sum('cases').sort('sum(cases)').show()

+-----------------------+----------+
|countriesAndTerritories|sum(cases)|
+-----------------------+----------+
|          Liechtenstein|      5472|
|                Iceland|     19708|
|                  Malta|     40721|
|             Luxembourg|     94511|
|                 Cyprus|    142228|
|                Finland|    207033|
|                Estonia|    229869|
|                 Latvia|    263145|
|                 Norway|    322717|
|               Slovenia|    443873|
|              Lithuania|    495686|
|                Denmark|    570502|
|                Ireland|    632177|
|                Croatia|    657726|
|               Bulgaria|    718651|
|                 Greece|   1012659|
|               Portugal|   1200193|
|                Hungary|   1208020|
|                Austria|   1226443|
|                 Sweden|   1238689|
+-----------------------+----------+
only showing top 20 rows



In [ ]:
covidSinNull.select('continentExp').distinct().show()

+------------+
|continentExp|
+------------+
|      Europe|
+------------+



In [ ]:
covidSinNull.select('countriesAndTerritories').distinct().show()

+-----------------------+
|countriesAndTerritories|
+-----------------------+
|                 Sweden|
|                Germany|
|                 France|
|                 Greece|
|               Slovakia|
|                Belgium|
|                Finland|
|                  Malta|
|                Croatia|
|                  Italy|
|              Lithuania|
|                 Norway|
|                  Spain|
|                Czechia|
|                Denmark|
|                Ireland|
|          Liechtenstein|
|                Iceland|
|                 Cyprus|
|                Estonia|
+-----------------------+
only showing top 20 rows



#***Consultas con SPARKSQL***


Venimos haciendo consultas de DataFrame, ahora vamos hacer consultas con Querys como si estubieramos usando Base de Datos. No lo puedo hacer en DataFrame, tengo que crear una vista o una tabla para hacer Querys.

In [ ]:
#Creamos una vista o una tabla de DataFrame de covid

covidSinNull.createOrReplaceTempView("table_covid")

In [ ]:
type('table_covid')

str

In [ ]:
##Muestre cada registro de casos el pais, el continente, el numero de casos, y la fecha de reporte, en casos mayores a 1000 en orden descendente
spark.sql('SELECT countriesAndTerritories, cases,dateRep, continentExp FROM table_covid WHERE cases > 1000 ORDER BY cases DESC').show()

+-----------------------+-------+----------+------------+
|countriesAndTerritories|  cases|   dateRep|continentExp|
+-----------------------+-------+----------+------------+
|                 France|3645305|01/03/2021|      Europe|
|                  Spain|3109689|28/02/2021|      Europe|
|                  Italy|2925265|01/03/2021|      Europe|
|                Germany|2447068|01/03/2021|      Europe|
|                 Poland|1706986|01/03/2021|      Europe|
|                Czechia|1240721|01/03/2021|      Europe|
|            Netherlands|1087822|01/03/2021|      Europe|
|               Portugal| 804562|01/03/2021|      Europe|
|                Romania| 801994|01/03/2021|      Europe|
|                Belgium| 777696|01/03/2021|      Europe|
|                 Sweden| 668966|01/03/2021|      Europe|
|               Slovakia| 592135|01/03/2021|      Europe|
|                Austria| 456112|01/03/2021|      Europe|
|                Hungary| 432925|01/03/2021|      Europe|
|             

In [ ]:
spark.sql('SELECT countriesAndTerritories AS Pais, sum(deaths) AS Muertes FROM table_covid GROUP BY countriesAndTerritories').show()

+-------------+-------+
|         Pais|Muertes|
+-------------+-------+
|       Sweden|  15204|
|      Germany| 106680|
|       France| 120832|
|       Greece|  19475|
|     Slovakia|  15516|
|      Belgium|  27685|
|      Finland|   1444|
|        Malta|    471|
|      Croatia|  11722|
|        Italy| 135049|
|    Lithuania|   7039|
|       Norway|   1141|
|        Spain|  88542|
|      Czechia|  34813|
|      Denmark|   3030|
|      Ireland|   5788|
|Liechtenstein|     68|
|      Iceland|     36|
|       Cyprus|    612|
|      Estonia|   1869|
+-------------+-------+
only showing top 20 rows



In [ ]:
df1 = spark.createDataFrame([(1, 'Julian'), (2 , 'Pepe'), (3, 'Maria'), (4, 'Carlos')], ["id","nombre"])
df2 = spark.createDataFrame([(1,21),(2,17),(3,43)],["id","edad"])

In [ ]:
dfJoin = df1.join(df2,'id', 'right')

In [ ]:
dfJoin.show()

+---+------+----+
| id|nombre|edad|
+---+------+----+
|  1|Julian|  21|
|  2|  Pepe|  17|
|  3| Maria|  43|
+---+------+----+



In [ ]:
dfJoin.createOrReplaceTempView('resultados')



In [ ]:
spark.sql('SELECT *FROM resultados WHERE edad >24 ORDER BY nombre').show()

+---+------+----+
| id|nombre|edad|
+---+------+----+
|  3| Maria|  43|
+---+------+----+

